In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis,skew

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import make_scorer
from tqdm import tqdm_notebook as tqdm
import pyeeg
from sklearn import svm
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from math import log, floor
from sklearn.impute import SimpleImputer
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, RandomForestRegressor, GradientBoostingClassifier )
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
import warnings  
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.feature_selection import mutual_info_classif

import yasa

In [2]:
xtrain_eeg1 = pd.read_csv("train_eeg1.csv").drop("Id", axis = 1)
xtrain_eeg2 = pd.read_csv("train_eeg2.csv").drop("Id", axis = 1)
xtrain_emg = pd.read_csv("train_emg.csv").drop("Id", axis = 1)

ytrain = pd.read_csv("train_labels.csv").drop("Id", axis = 1)

xtest_eeg1 = pd.read_csv("test_eeg1.csv").drop("Id", axis = 1)
xtest_eeg2 = pd.read_csv("test_eeg2.csv").drop("Id", axis = 1)
xtest_emg = pd.read_csv("test_emg.csv").drop("Id", axis = 1)

xtrain_eeg1 = xtrain_eeg1.values
xtrain_eeg2 = xtrain_eeg2.values
xtrain_emg = xtrain_emg.values

xtest_eeg1 = xtest_eeg1.values 
xtest_eeg2 = xtest_eeg2.values
xtest_emg = xtest_emg.values

In [17]:
def _log_n(min_n, max_n, factor):
    """
    Creates a list of integer values by successively multiplying a minimum
    """
    max_i = int(floor(log(1.0 * max_n / min_n) / log(factor)))
    ns = [min_n]
    for i in range(max_i + 1):
        n = int(floor(min_n * (factor ** i)))
        if n > ns[-1]:
            ns.append(n)
    return np.array(ns, dtype=np.int64)

##########################################################################

def _linear_regression(x, y):
    """Fast linear regression using Numba.
    Parameters
    ----------
    x, y : ndarray, shape (n_times,)
        Variables
    Returns
    -------
    slope : float
        Slope of 1D least-square regression.
    intercept : float
        Intercept
    """
    n_times = x.size
    sx2 = 0
    sx = 0
    sy = 0
    sxy = 0
    for j in range(n_times):
        sx2 += x[j] ** 2
        sx += x[j]
        sxy += x[j] * y[j]
        sy += y[j]
    den = n_times * sx2 - (sx ** 2)
    num = n_times * sxy - sx * sy
    slope = num / den
    intercept = np.mean(y) - slope * np.mean(x)
    return slope, intercept

##########################################################################



def _dfa(x):
    """
    Utility function for detrended fluctuation analysis
    """
    N = len(x)
    nvals = _log_n(4, 0.1 * N, 1.2)
    walk = np.cumsum(x - x.mean())
    fluctuations = np.zeros(len(nvals))

    for i_n, n in enumerate(nvals):
        d = np.reshape(walk[:N - (N % n)], (N // n, n))
        ran_n = np.array([float(na) for na in range(n)])
        d_len = len(d)
        slope = np.empty(d_len)
        intercept = np.empty(d_len)
        trend = np.empty((d_len, ran_n.size))
        for i in range(d_len):
            slope[i], intercept[i] = _linear_regression(ran_n, d[i])
            y = np.zeros_like(ran_n)
            # Equivalent to np.polyval function
            for p in [slope[i], intercept[i]]:
                y = y * ran_n + p
            trend[i, :] = y
        # calculate standard deviation (fluctuation) of walks in d around trend
        flucs = np.sqrt(np.sum((d - trend) ** 2, axis=1) / n)
        # calculate mean fluctuation over all subsequences
        fluctuations[i_n] = flucs.sum() / flucs.size

    # Filter zero
    nonzero = np.nonzero(fluctuations)[0]
    fluctuations = fluctuations[nonzero]
    nvals = nvals[nonzero]
    if len(fluctuations) == 0:
        # all fluctuations are zero => we cannot fit a line
        dfa = np.nan
    else:
        dfa, _ = _linear_regression(np.log(nvals), np.log(fluctuations))
    return dfa

##########################################################################


def matrix_dfa(x):
    dfa_ = np.zeros((x.shape[0],), dtype=np.float64)
    for i in (np.arange(x.shape[0])):
        dfa_[i] = _dfa(np.asarray(x[i], dtype=np.float64))
    return np.reshape(dfa_, (-1, 1))

##########################################################################

def _embed(x, order=3, delay=1):
    """ VECTORIZED!! Kind of TESTED. Time-delay embedding. x is an (nxd) matrix
    """
    N = x.shape[1]
    if order * delay > N:
        raise ValueError("Error: order * delay should be lower than x.size")
    if delay < 1:
        raise ValueError("Delay has to be at least 1.")
    if order < 2:
        raise ValueError("Order has to be at least 2.")
    Y = np.zeros((x.shape[0], N - (order - 1) * delay, order))
    for i in range(order):
        Y[:, :, i] = x[:, i * delay:i * delay + Y.shape[1]]
    return Y

##########################################################################

def hfd(X, Kmax):
    """ VECTORIZED!!! TESTED: Matches the for loop output. Can test easily comparing
    to the pyeeg.hfd() function. X now a (nxd) matrix.
    Compute Higuchi Fractal Dimension of a time series X. kmax
     is an HFD parameter
    """
    L = []
    x = []
    N = X.shape[1]
    for k in (range(1, Kmax)):
        # Lk = np.empty(shape=[0, ])
        Lk = np.empty(shape=[X.shape[0], 1])
        for m in range(0, k):
            Lmk = 0
            for i in range(1, int(np.floor((N - m) / k))):
                Lmk += np.abs(X[:, m + i * k] - X[:, m + i * k - k])
            Lmk = Lmk * (N - 1) / np.floor((N - m) / float(k)) / k
            Lmk = np.reshape(Lmk, (Lmk.shape[0], 1))
            Lk = np.hstack((Lk, Lmk))

        # Remove that first placeholder column of zeros in Lk:
        Lk = Lk[:, 1:]
        L.append(np.log(np.mean(Lk, axis=1)))
        x.append([np.log(float(1) / k), 1]) # Fix this!!!

    (p, _, _, _) = np.linalg.lstsq(x, L)
    return p[0]

##########################################################################

def pfd(X):
    """VECTORIZED!!! TESTED, matches the 1d time series output. Now accepts (nxd) matrices as input
    Compute Petrosian Fractal Dimension of a time series from either two
    cases below:
        1. X, the time series of type list (default)
        2. D, the first order differential sequence of X (if D is provided,
           recommended to speed up)
    In case 1, D is computed using Numpy's difference function.
    To speed up, it is recommended to compute D before calling this function
    because D may also be used by other functions whereas computing it here
    again will slow down.
    """
    n = X.shape[1]
    diff = np.diff(X, axis=1)
    N_delta = np.sum(diff[:, 1:-1] * diff[:, 0:-2] < 0, axis=1)
    return np.log10(n) / (np.log10(n) + np.log10(n / (n + 0.4 * N_delta)))

##########################################################################

def fisher_info(X, Tau=3, DE=1, W=None):
    """ VECTORIZED, TESTED, gives approximate results but not exact. Compute SVD Entropy from either two cases below:
    """
    if W is None:
        Y = _embed(X, Tau, DE)
        W = np.linalg.svd(Y, compute_uv=0)
        W = np.divide(W, np.reshape(np.sum(W, axis=1), (-1, 1)))  # normalize singular values
    return -1 * np.sum(W * np.log(W), axis=1)

##########################################################################

def extract_bandpower_eeg(signal, frequency = 128):
    for i in (np.arange(signal.shape[0] / 100) + 1):
        if i == 1:
            df = yasa.bandpower(signal[0:int(100*i),:], sf=frequency)
        else:
            df = df.append(yasa.bandpower(signal[int(100*(i-1)):int(100*i),:], sf=frequency))
    
    df = df.set_index(np.arange(signal.shape[0]))
    df = df.drop(columns = ["FreqRes","Relative"], axis = 1)
    return df

##########################################################################

'''input must be np.ndarray'''

def vectorized_adv_stat(signal, fs=128):
    K_boundary = 10         # to be tuned
    t_fisher = 12          # to be tuned
    d_fisher = 40          # to be tuned
    features_num = 11
    threshold =  0.0009
    advanced_stats = np.zeros((signal.shape[0],features_num))
    # Missing fisher info and dfa
    feat_array = np.array([
                           fisher_info(signal, t_fisher, d_fisher),
                           pfd(signal),
                           hfd(signal, K_boundary),
                           np.sum((np.power(np.abs(signal),(-0.3)) > 20), axis=1),
                           np.sum((np.abs(signal)) > threshold, axis=1),
                           np.std(np.power(np.abs(signal),(0.05)), axis=1),
                           np.sqrt(np.mean(np.power(np.diff(signal, axis=1), 2), axis=1)),
                           np.mean(np.abs(np.diff(signal, axis=1)), axis=1),
                           np.mean(np.power(signal, 5), axis=1),
                           np.sum(np.power(signal, 2), axis=1)
                           ]).T
    #feat_array = np.concatenate((feat_array, matrix_dfa(signal)), axis=1) # Concatenate the lengthy dfa calculation

    return feat_array

##########################################################################

def simple_stats(signal, fs = 128):
           
    if (len(signal.shape) > 1) and (signal.shape[1]!=1):
        simple_stats = np.array([np.mean(signal, axis=1), 
                        np.median(signal, axis=1),
                        np.std(signal, axis=1), 
                        np.max(signal, axis=1),
                        np.min(signal, axis=1), 
                        kurtosis(signal, axis=1),
                        skew(signal, axis=1), 
                        np.sum(np.abs(signal), axis = 1)]).T
                        
    else:
        print("Not Tested with this input!")
        simple_stats =  np.array([np.mean(signal), 
                        np.median(signal), 
                        np.std(sigal),
                        np.max(signal), 
                        np.min(signal), 
                        float(kurtosis(signal)),
                        float(skew(signal))])
        
    

    return (simple_stats)

##########################################################################

def EEG_feature_extraction(signal, fs = 128): 
    eeg_features = np.concatenate((extract_bandpower_eeg(signal, frequency = 128), vectorized_adv_stat(signal, fs=128),
                                   simple_stats(signal, fs = 128)), axis = 1)
    
    return(eeg_features)

##########################################################################

def EMG_feature_extraction(signal, fs = 128):
    
    features_num_emg = 6
    
    if (len(signal.shape) > 1) and (signal.shape[1]!=1):
        simple_stats = np.array([np.mean(signal, axis=1), 
                        np.median(signal, axis=1),
                        np.std(signal, axis=1), 
                        np.max(signal, axis=1),
                        np.min(signal, axis=1), 
                        kurtosis(signal, axis=1),
                        skew(signal, axis=1), 
                        pd.Series(np.sum(np.abs(signal), axis = 1))]).T
                        
    else:
        print("Not Tested with this input!")
        simple_stats =  np.array([np.mean(signal), 
                        np.median(signal), 
                        np.std(sigal),
                        np.max(signal), 
                        np.min(signal), 
                        float(kurtosis(signal)),
                        float(skew(signal))])

    advanced_stats = np.zeros((signal.shape[0],features_num_emg))
    for i in tqdm((np.arange(signal.shape[0]))):
        feat_array = np.array([
                              np.median(signal[i,:] ** 2), 
                              np.median(np.abs(np.diff(signal[i,:]))), 
                              np.std(np.abs(np.diff(signal[i,:]))), 
                              np.sum(np.abs(np.diff(signal[i,:]))), 
                              np.mean(np.power(np.diff(signal[i,:]), 2)),
                              np.std(abs(signal[i,:]))
                            ])

        advanced_stats[i, :] = feat_array 
        
        union_smpl_adv = np.concatenate((simple_stats, advanced_stats), axis = 1)

    return(union_smpl_adv)

##########################################################################

'''change for submissions predictions'''

def hard_voter(ypred_crf, ypred_svm, 
               ypred_logi, ypred_KNN,
               ypred_RNN, ypred_SGD):
    
    predictions = np.zeros(xtest.shape[0])   
    
    for i in range(xtest.shape[0]):

        counts = np.bincount(np.array([ypred_crf[i],ypred_svm[i], 
                    ypred_logi[i], ypred_KNN[i], ypred_RNN[i], ypred_SGD[i]]))
        predictions[i] = np.argmax(counts)
        
    return(np.asarray(predictions, dtype=np.int32)) 


'''if (ypred_crf[i] != ypred_boost[i] and  
ypred_crf[i] != ypred_svm[i] and
ypred_svm[i] != ypred_boost[i]): 

rand = np.random.choice(np.arange(1, 4), p=[p_crf,p_boost,p_svm])

predictions[i] = rand'''

'if (ypred_crf[i] != ypred_boost[i] and  \nypred_crf[i] != ypred_svm[i] and\nypred_svm[i] != ypred_boost[i]): \n\nrand = np.random.choice(np.arange(1, 4), p=[p_crf,p_boost,p_svm])\n\npredictions[i] = rand'

# Extract Features --- Full Data

In [18]:
xtrain_eeg1_processed = EEG_feature_extraction(xtrain_eeg1, 128)
xtrain_eeg2_processed = EEG_feature_extraction(xtrain_eeg2, 128)
xtrain_emg_processed = EMG_feature_extraction(xtrain_emg, 128)
xtrain = np.concatenate((xtrain_eeg1_processed, 
                         xtrain_eeg2_processed, 
                         xtrain_emg_processed), 
                         axis = 1)

xtest_eeg1_processed = EEG_feature_extraction(xtest_eeg1, 128)
xtest_eeg2_processed = EEG_feature_extraction(xtest_eeg2, 128)
xtest_emg_processed = EMG_feature_extraction(xtest_emg, 128)
xtest = np.concatenate((xtest_eeg1_processed, 
                         xtest_eeg2_processed, 
                         xtest_emg_processed), 
                         axis = 1)

print(xtrain.shape, xtest.shape)

xtrain_grid = pd.DataFrame(xtrain)
scaler = preprocessing.StandardScaler()
xtrain_rescaled = scaler.fit_transform(xtrain_grid)
xtest_rescaled = scaler.fit_transform(xtest)


(64800, 60) (43200, 60)


# calculate mutual information of features 

In [380]:
mutual_info = mutual_info_classif(xtrain, ytrain, discrete_features='auto', n_neighbors = 10, copy=True, random_state = 15)

In [21]:
#leave_out_feat = np.where(mutual_info > 0.08)
#leave_out_feat = list(leave_out_feat)[0]

'''update xtrain and xtest'''

xtrain_filter = xtrain#[:,leave_out_feat]
xtest_filter = xtest#[:,leave_out_feat]

# GS SVM  --- OPT DONE

In [349]:
xtrain_grid = pd.DataFrame(xtrain)

steps = [("scaler", StandardScaler()), ("classifier", SVC())]
pipeline = Pipeline(steps = steps)

parameters = {"classifier__kernel": ["rbf"],
              "classifier__gamma": ["auto"],
              "classifier__C": [0.01, 0.1,0.3],  
              "classifier__class_weight": ["balanced"]
             }
grid_svm = GridSearchCV(pipeline, parameters, cv = 3, scoring = 'balanced_accuracy', verbose = 1)

grid_svm.fit(xtrain_grid, ytrain.values.ravel())
print(grid_svm.best_score_)
print(grid_svm.best_params_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] classifier__C=0.01, classifier__class_weight=balanced, classifier__gamma=auto, classifier__kernel=rbf 
[CV]  classifier__C=0.01, classifier__class_weight=balanced, classifier__gamma=auto, classifier__kernel=rbf, total=  36.7s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   36.6s remaining:    0.0s


[CV] classifier__C=0.01, classifier__class_weight=balanced, classifier__gamma=auto, classifier__kernel=rbf 
[CV]  classifier__C=0.01, classifier__class_weight=balanced, classifier__gamma=auto, classifier__kernel=rbf, total=  48.5s
[CV] classifier__C=0.01, classifier__class_weight=balanced, classifier__gamma=auto, classifier__kernel=rbf 
[CV]  classifier__C=0.01, classifier__class_weight=balanced, classifier__gamma=auto, classifier__kernel=rbf, total=  47.1s
[CV] classifier__C=0.1, classifier__class_weight=balanced, classifier__gamma=auto, classifier__kernel=rbf 
[CV]  classifier__C=0.1, classifier__class_weight=balanced, classifier__gamma=auto, classifier__kernel=rbf, total=  18.0s
[CV] classifier__C=0.1, classifier__class_weight=balanced, classifier__gamma=auto, classifier__kernel=rbf 
[CV]  classifier__C=0.1, classifier__class_weight=balanced, classifier__gamma=auto, classifier__kernel=rbf, total=  29.6s
[CV] classifier__C=0.1, classifier__class_weight=balanced, classifier__gamma=aut

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  4.6min finished


0.919127323819208
{'classifier__C': 0.01, 'classifier__class_weight': 'balanced', 'classifier__gamma': 'auto', 'classifier__kernel': 'rbf'}


# GS Gradient Boosting Method (very slow search)  --- MANUAL OPT

In [350]:
'''
Gradient Boosting APPROACH -- GRID-SEARCH CV
''' 
xtrain_grid = pd.DataFrame(xtrain)


steps = [("scaler", preprocessing.StandardScaler()), 
         ("classifier", GradientBoostingClassifier())]
pipeline = Pipeline(steps = steps)

parameters = {"classifier__max_depth": [20],
              "classifier__n_estimators": [100],
              "classifier__learning_rate": [0.2],
              "classifier__max_features": [20]
             }

grid_boost = GridSearchCV(pipeline, parameters, cv = 3,scoring = 'balanced_accuracy', verbose = 1)
grid_boost.fit(xtrain_grid, ytrain.values.ravel())

print(grid_boost.best_score_)
print(grid_boost.best_params_)

'\nGradient Boosting APPROACH -- GRID-SEARCH CV\n'

# GS Logistic  --- OPT DONE 

In [344]:
steps = [("scaler", StandardScaler()), ("classifier", LogisticRegression(fit_intercept=True))]
pipeline = Pipeline(steps = steps)

parameters = {"classifier__multi_class": ["auto"],
              "classifier__C": [0.02,0.01],  
              "classifier__class_weight": ["balanced"], 
              "classifier__max_iter": [10,15,20],
              "classifier__verbose": [0],
              "classifier__penalty": ['l2'],
              "classifier__solver": ['liblinear']
             }
grid_logi = GridSearchCV(pipeline, parameters, cv = 3, scoring = 'balanced_accuracy', verbose = 1)

grid_logi.fit(xtrain_grid, ytrain.values.ravel())
print(grid_logi.best_score_)
print(grid_logi.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:   24.6s finished


0.9267708371037586
{'classifier__C': 0.01, 'classifier__class_weight': 'balanced', 'classifier__max_iter': 10, 'classifier__multi_class': 'auto', 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear', 'classifier__verbose': 0}


# GS Random Forest  --- OPT DONE

In [345]:
steps = [("scaler", StandardScaler()), ("classifier", RandomForestClassifier(criterion='entropy', max_features='auto'))]
pipeline = Pipeline(steps = steps)

parameters = {"classifier__n_estimators": [20],
              "classifier__max_depth": [5], 
              "classifier__class_weight": ["balanced"], 
              "classifier__min_samples_split":[100,200], 
              "classifier__min_samples_leaf": [100]
             }
grid_RandFor = GridSearchCV(pipeline, parameters, cv = 3, scoring = 'balanced_accuracy', verbose = 1)

grid_RandFor.fit(xtrain_grid, ytrain.values.ravel())
print(grid_RandFor.best_score_)
print(grid_RandFor.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   12.0s finished


0.8849094425926666
{'classifier__class_weight': 'balanced', 'classifier__max_depth': 5, 'classifier__min_samples_leaf': 100, 'classifier__min_samples_split': 100, 'classifier__n_estimators': 20}


# GS KNN (Very slow search)  --- OPT DONE

In [111]:
xtrain_grid = pd.DataFrame(xtrain)

steps = [("scaler", preprocessing.StandardScaler()), 
         ("classifier", KNeighborsClassifier())]
pipeline = Pipeline(steps = steps)

parameters = {"classifier__n_neighbors": [5,10,20],
              "classifier__weights": ['uniform'],
              "classifier__leaf_size": [200]
             }

grid_KNN = GridSearchCV(pipeline, parameters, cv = 3, scoring = 'balanced_accuracy', verbose = 1)
grid_KNN.fit(xtrain_grid, ytrain.values.ravel())

print(grid_KNN.best_score_)
print(grid_KNN.best_params_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 15.0min finished


0.9065325243946395
{'classifier__leaf_size': 200, 'classifier__n_neighbors': 20, 'classifier__weights': 'uniform'}


# GS SGD Classifier --- OPT DONE

In [348]:
xtrain_grid = pd.DataFrame(xtrain)

steps = [("scaler", preprocessing.StandardScaler()), 
         ("classifier", SGDClassifier(shuffle = True, fit_intercept=True))]
pipeline = Pipeline(steps = steps)

parameters = {"classifier__penalty": ['l2'],
              "classifier__alpha": [0.0001],
              "classifier__max_iter": [100],
              "classifier__loss": ['hinge'],
              "classifier__learning_rate": ['optimal'],
              "classifier__power_t":[2]
              }

grid_SGD = GridSearchCV(pipeline, parameters, cv = 3, scoring = 'balanced_accuracy', verbose = 1)
grid_SGD.fit(xtrain_grid, ytrain.values.ravel())

print(grid_SGD.best_score_)
print(grid_SGD.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.6s finished


0.8601686473634104
{'classifier__alpha': 0.0001, 'classifier__learning_rate': 'optimal', 'classifier__loss': 'hinge', 'classifier__max_iter': 100, 'classifier__penalty': 'l2', 'classifier__power_t': 2}


# Running Validation --- train on subj 1,2 --- test on 3

In [280]:
'''extracting features'''

subj12_eeg1 = xtrain_eeg1[0:43200,:]
subj12_eeg2 = xtrain_eeg2[0:43200,:]
subj12_emg = xtrain_emg[0:43200,:]
ytrain_subj12 = np.reshape(ytrain.values, (-1,))[0:43200]

xtrain_subj12_eeg_1 = EEG_feature_extraction(subj12_eeg1, 128)  
xtrain_subj12_eeg_2 = EEG_feature_extraction(subj12_eeg2, 128)
xtrain_subj12_emg = EMG_feature_extraction(subj12_emg, 128)
xtrain_subj12 = np.concatenate((xtrain_subj12_eeg_1, xtrain_subj12_eeg_2, xtrain_subj12_emg), axis = 1)


'''creating validation data'''

subj3_eeg1 = xtrain_eeg1[43200:64800,:]
subj3_eeg2 = xtrain_eeg2[43200:64800,:]
subj3_emg = xtrain_emg[43200:64800,:]
ytest_subj3 = np.reshape(ytrain.values, (-1,))[43200:64800]

xtest_subj3_eeg_1 = EEG_feature_extraction(subj3_eeg1, 128)
xtest_subj3_eeg_2 = EEG_feature_extraction(subj3_eeg2, 128)
xtest_subj3_emg = EMG_feature_extraction(subj3_emg, 128)
xtest_subj3 = np.concatenate((xtest_subj3_eeg_1, xtest_subj3_eeg_2, xtest_subj3_emg), axis = 1)

xtrain_subj12_rescaled = preprocessing.StandardScaler().fit_transform(xtrain_subj12)
xtest_subj3_rescaled = preprocessing.StandardScaler().fit_transform(xtest_subj3)

In [293]:
### '''Testing the models'''
### 
### '''SVM'''
### 
### clf_svm = svm.SVC(kernel='rbf', C = 0.01, gamma = 'auto', class_weight = 'balanced')
### clf_svm.fit(xtrain_subj12, ytrain_subj12)
### ypred_svm = clf_svm.predict(xtest_subj3)
### print('SVM done')
### 
### 
### '''Multinomial Logistic Regression'''
### 
### clf_logi = LogisticRegression(random_state=10, C = 0.01, class_weight = 'balanced', max_iter = 10, 
###                               multi_class='auto', penalty = 'l2', solver = 'liblinear')
### clf_logi.fit(xtrain_subj12, ytrain_subj12)
### ypred_logi = clf_logi.predict(xtest_subj3)
### print('Multinomial Logistic done')
### 
### 
### 
### '''SGD'''
### 
### clf_SGD = SGDClassifier(alpha = 0.0001, learning_rate = 'optimal', loss = 'hinge', max_iter = 1000, 
###                         penalty = 'l1', power_t = 2, shuffle = True, fit_intercept=True)
### clf_SGD.fit(xtrain_subj12, ytrain_subj12)
### ypred_SGD = clf_SGD.predict(xtest_subj3)
### print('SGD done')
### 
### 
### 
### '''Boosting Classifier'''
### 
### clf_boost = GradientBoostingClassifier(n_estimators = 200, max_depth = 8, 
###                                        learning_rate = 0.1, max_features = 20)
### clf_boost.fit(xtrain_subj12, ytrain_subj12)
### ypred_boost = clf_boost.predict(xtest_subj3)
### print('Gradient Boosting done')
### 
### 
### 
### '''Random Forest'''
### 
### clf_RandFor = RandomForestClassifier(max_depth=5, random_state=1, class_weight = 'balanced', 
###                          min_samples_leaf = 100, min_samples_split = 100, n_estimators = 20)
### clf_RandFor.fit(xtrain_subj12, ytrain_subj12)
### ypred_RandFor = clf_RandFor.predict(xtest_subj3)
### print('Random Forest Done')
### 
### 
### '''KNN'''
### 
### clf_KNN = KNeighborsClassifier(leaf_size = 200, n_neighbors = 20, weights = 'uniform')
### clf_KNN.fit(xtrain_subj12, ytrain_subj12)
### ypred_KNN = clf_KNN.predict(xtest_subj3)
### print('KNN done')
### 
### 
### '''SGD'''
### 
### clf_SGD = SGDClassifier(alpha = 0.0001, learning_rate = 'optimal', loss = 'hinge',
###                        max_iter = 100, penalty = 'l2', power_t = 2, shuffle = True)
### clf_SGD.fit(xtrain_subj12, ytrain_subj12)
### ypred_SGD = clf_SGD.predict(xtest_subj3)
### print('SGD done')
### 
### 
### '''CRF'''
### 
### ypred_crf = pd.read_csv("CRFc0.1w0m2000_subj3pred.csv").drop("Id", axis = 1)
### ypred_crf = np.reshape(ypred_crf.values, (-1,))
### print('CRF Done')
### 

SVM done
Multinomial Logistic done
SGD done
Gradient Boosting done
Random Forest Done
KNN done
SGD done
CRF Done


IndexError: index 0 is out of bounds for axis 0 with size 0

In [304]:
#################################################################################

y_val = hard_voter(ypred_crf, ypred_boost, ypred_svm, ypred_logi, ypred_RandFor, ypred_KNN, ypred_SGD) 
BMAC = balanced_accuracy_score(ytest_subj3, y_val)

print(BMAC)

0.8216955827937881


# Prediction Vectors

In [26]:
'''load DFA data'''

dfa_xtrain_eeg1 = pd.read_csv('dfa_xtrain_eeg1.csv', header = None)
#dfa_xtrain_eeg1 = np.reshape(dfa_xtrain_eeg1.values, (-1,))

dfa_xtrain_eeg2 = pd.read_csv('dfa_xtrain_eeg2.csv', header = None)
#dfa_xtrain_eeg2 = np.reshape(dfa_xtrain_eeg2.values, (-1,))

dfa_xtrain_emg = pd.read_csv('dfa_xtrain_emg.csv', header = None)
#dfa_xtrain_emg = np.reshape(dfa_xtrain_emg.values, (-1,))

dfa_xtest_eeg1 = pd.read_csv('dfa_xtest_eeg1.csv', header = None)
#dfa_xtest_eeg1 = np.reshape(dfa_xtest_eeg1.values, (-1,))

dfa_xtest_eeg2 = pd.read_csv('dfa_xtest_eeg2.csv', header = None)
#dfa_xtest_eeg2 = np.reshape(dfa_xtest_eeg2.values, (-1,))

dfa_xtest_emg = pd.read_csv('dfa_xtest_emg.csv', header = None)
#dfa_xtest_emg = np.reshape(dfa_xtest_emg.values, (-1,))




'''update xtrain, xtest'''

xtrain_comp = np.concatenate((xtrain_filter, dfa_xtrain_eeg1, dfa_xtrain_eeg2, dfa_xtrain_emg), axis = 1)
xtest_comp = np.concatenate((xtest_filter, dfa_xtest_eeg1, dfa_xtest_eeg2, dfa_xtest_emg), axis = 1)

'''standardise'''

xtrain_rescaled = preprocessing.StandardScaler().fit_transform(xtrain_comp)
xtest_rescaled = preprocessing.StandardScaler().fit_transform(xtest_comp)

In [27]:
dfa_xtest_eeg1.shape

(43200, 1)

In [9]:
ypred_svm.shape

NameError: name 'ypred_svm' is not defined

In [28]:
'''SVM'''
clf_svm = svm.SVC(kernel='rbf', C = 0.01, gamma = 'auto', class_weight = 'balanced')
clf_svm.fit(xtrain_rescaled, ytrain)
ypred_svm = clf_svm.predict(xtest_rescaled)
print('SVM done')


'''Gradient Boosting'''
clf_boost = GradientBoostingClassifier(n_estimators = 200, max_depth = 8, learning_rate = 0.1, max_features = 20)
clf_boost.fit(xtrain_rescaled, ytrain)
ypred_boost = clf_boost.predict(xtest_rescaled)
print('Gradient Boosting done')


'''Multinomial Logistic'''
clf_logi = LogisticRegression(random_state=10, C = 0.01, class_weight = 'balanced', max_iter = 10, 
                              multi_class='auto', penalty = 'l2', solver = 'liblinear')
clf_logi.fit(xtrain_rescaled, ytrain)
ypred_logi = clf_logi.predict(xtest_rescaled)
print('Multinomial Logistic done')

'''Random Forest'''
clf_RandFor = RandomForestClassifier(max_depth=5, random_state=1, class_weight = 'balanced', 
                         min_samples_leaf = 100, min_samples_split = 100, n_estimators = 20)
clf_RandFor.fit(xtrain_rescaled, ytrain)
ypred_RandFor = clf_RandFor.predict(xtest_rescaled)
print('Random Forest Done')


'''KNN'''
clf_KNN = KNeighborsClassifier(leaf_size = 200, n_neighbors = 20, weights = 'uniform')
clf_KNN.fit(xtrain_rescaled, ytrain)
ypred_KNN = clf_KNN.predict(xtest_rescaled)
print('KNN done')


'''SGD'''
clf_SGD = SGDClassifier(alpha = 0.0001, learning_rate = 'optimal', loss = 'hinge',
                       max_iter = 100, penalty = 'l2', power_t = 2, shuffle = True)
clf_SGD.fit(xtrain_rescaled, ytrain)
ypred_SGD = clf_SGD.predict(xtest_rescaled)
print('SGD done')


'''CRF'''
ypred_crf = pd.read_csv("ypred_crf.csv").drop("Id", axis = 1)
ypred_crf = np.reshape(ypred_crf.values, (-1,))
print('CRF Done')

'''RNN'''
ypred_RNN = pd.read_csv("RNN_13_32_0.1_0.2_a_32.csv").drop("Id", axis = 1)
ypred_RNN = np.reshape(ypred_RNN.values, (-1,))
print('RNN Done')

SVM done
Gradient Boosting done
Multinomial Logistic done
Random Forest Done
KNN done
SGD done
CRF Done
RNN Done


# Hard Classifier -- Submission

In [30]:
submission = hard_voter(ypred_crf, ypred_svm, ypred_logi, ypred_KNN, ypred_RNN, ypred_SGD)
index = pd.read_csv("sample.csv")
index['y'] = submission
index.to_csv("hard_voter.csv")